In [15]:
import webbrowser
import requests, bs4
import pandas as pd
import proxyscrape
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from proxyscrape import create_collector, get_collector

### Proxy Scraping Function:

In [16]:
def scrape_proxies(collector, timeout_filtering, url_testing):
    
    #Refresh list of proxies to scrape 
    collector.refresh_proxies(force=True)
    
    #Scrape HTTPS proxies that provide anonymity for specific countries. See proxyscrape documentation for more info.
#     proxy_list = collector.get_proxy({'code': ('us'), 'anonymous': True, 'type': 'https'}) #Get one USA proxy
#     proxy_list = collector.get_proxies({'code': ('us'), 'anonymous': True, 'type': 'https'}) #Get ALL USA proxies
    proxy_list = collector.get_proxies({'code': ('us', 'ca'), 'anonymous': True, 'type': 'https'}) #Get ALL USA + Canadian proxies
#     proxy_list = collector.get_proxies({'code': ('us', 'ca', 'de', 'fr'), 'anonymous': True, 'type': 'https'}) ALL USA+Canada+German+French
    
    #Let's extract useful information from this scraped list of proxies
    #Note that scraped list may contain duplicates
    
    ip_addresses = []
    ports = []
    countries = []

    #Collecting proxy information
    for proxy in proxy_list:
        if proxy[0] in ip_addresses and proxy[1] in ports: #preventing duplicates from being recorded
            continue

        ip_addresses.append(proxy[0]) #Collect the ip addresses
        ports.append(proxy[1]) #Collect the port numbers
        countries.append(proxy[3]) #Collect their corresponding countries
    
    #Now let's filter out the working (good) proxies
    
    good_proxies = [] 
    good_proxies_country = []

    for ip_address, port, country in zip(ip_addresses, ports, countries):
        full_proxy = ip_address + ":" + port

        proxy_temp = {"https": "https://" + full_proxy} #putting it in correct format
        print("Trying Proxy: " + "https://" + full_proxy)

        #Filtering proxies
        try:
            requests.get(url_testing, proxies=proxy_temp, timeout=timeout_filtering) #Testing proxy by downloading content in URL with a timeout 
            print("Success! Added to List.") #Successful downloading means that the proxy works
            good_proxies.append(full_proxy)
            good_proxies_country.append(country)
        except:
            print("Connection error. Trying next proxy.")
    
    return good_proxies, good_proxies_country

### Initialize Proxy Collection here:

In [18]:
url_testing = 'http://www.foo.com/' #URL we'll use to check if a proxy works. Put your desired url to scrape here for best results.
timeout_filtering = 4 #seconds, the timer we'll set on checking url_testing with proxies

### We will use the 'proxyscrape' module for scraping proxies. Documentation here: https://pypi.org/project/proxyscrape/ ###

#Initialize collector for proxies. RUN THIS ONCE.
collector = create_collector('my-collector', 'https')

#Retrieve a collector if already initialized
# collector = get_collector('my-collector')

#Outputs a list of working proxies and their corresponding geolocation (country)
good_proxies, good_proxies_country  = scrape_proxies(collector, timeout_filtering, url_testing)

Trying Proxy: https://3.135.90.78:17567
Success! Added to List.
Trying Proxy: https://166.70.207.2:8118
Success! Added to List.
Trying Proxy: https://107.178.4.215:35330
Success! Added to List.
Trying Proxy: https://104.45.188.43:3128
Success! Added to List.
Trying Proxy: https://50.192.195.69:52018
Success! Added to List.
Trying Proxy: https://54.214.52.181:80
Success! Added to List.
Trying Proxy: https://24.172.34.114:49920
Success! Added to List.
Trying Proxy: https://104.236.48.178:8080
Success! Added to List.
Trying Proxy: https://198.204.253.114:3128
Success! Added to List.
Trying Proxy: https://50.233.228.147:8080
Success! Added to List.
Trying Proxy: https://144.217.101.245:3129
Success! Added to List.
Trying Proxy: https://34.105.59.26:80
Success! Added to List.
Trying Proxy: https://162.17.252.5:43764
Success! Added to List.
Trying Proxy: https://160.2.38.41:8080
Success! Added to List.
Trying Proxy: https://192.3.31.67:9998
Success! Added to List.
Trying Proxy: https://173.8

#### Display collected proxies in dataframe

In [19]:
pd.DataFrame(list(zip(good_proxies, good_proxies_country)), 
               columns =['Proxy', 'Country']) 

,Proxy,Country
0,3.135.90.78:17567,united states
1,166.70.207.2:8118,united states
2,107.178.4.215:35330,united states
3,104.45.188.43:3128,united states
4,50.192.195.69:52018,united states
5,54.214.52.181:80,united states
6,24.172.34.114:49920,united states
7,104.236.48.178:8080,united states
8,198.204.253.114:3128,united states
9,50.233.228.147:8080,united states


### Robust downloading Function with Requests:

In [22]:
def robust_download(url, current_proxy, good_proxies, timeout_req):
    
    #Put current/preferred proxy on top of list for iteration
    if current_proxy in good_proxies:
        good_proxies.insert(0, good_proxies.pop(good_proxies.index(current_proxy)))
    else:
        good_proxies.insert(0,current_proxy) 
        
    for proxy in good_proxies:
        
        print("===================== DOWNLOADING =====================")
        
        try:
            if proxy == current_proxy:
                print("Current Proxy: " + proxy)
            else:
                print("Trying proxy: " + proxy)
                
            #download webpage
            req = requests.get(url, proxies={"https": "https://" + current_proxy}, timeout=timeout_req)
            req.raise_for_status() #Throw an error if the timer runs out or there's a failure in downloading
            
            #Proxy works!
            if proxy == current_proxy:
                print("Still works: " + proxy)
            else:
                print("Found new working proxy: " + proxy)
                
            checker = True
            current_proxy = proxy
            break
        except:
            checker = False
            print("Connection error. Trying next proxy.")
            continue 
            
    if checker == False:
        raise Exception("No working proxies!") #Error thrown if none of the proxies are working
    else:
        print("Final proxy: " + current_proxy)
        print("Download Successful!")
        
    return current_proxy #return new working proxy

### Robust Requests downloading example:

In [23]:
#We can start our download with an initial/preferred proxy
preferred_proxy = random.choice(good_proxies) #Just picking a random proxy from list

url = 'http://www.foo.com/' #url you want to download webpages from
timeout_req = 30 #seconds, the timer set on checking if you can download the pages with a proxy

#Instead of requests.get(url), you will run this:
preferred_proxy = robust_download(url, preferred_proxy, good_proxies, timeout_req) 

===================== DOWNLOADING =====================
Current Proxy: 198.55.103.233:80
Still works: 198.55.103.233:80
Final proxy: 198.55.103.233:80
Download Successful!
